In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('adult_income.csv')

## 1. Clear whitespace

In [ ]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

## 2. Missing values

In [ ]:
df = df.applymap(lambda x: np.nan if x == '?' else x)

In [ ]:
df.isna().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation        1843
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64

In [ ]:
df['workclass'].value_counts()

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: workclass, dtype: int64

In [ ]:
df['native-country'].value_counts()

United-States                 29170
Mexico                          643
Philippines                     198
Germany                         137
Canada                          121
Puerto-Rico                     114
El-Salvador                     106
India                           100
Cuba                             95
England                          90
Jamaica                          81
South                            80
China                            75
Italy                            73
Dominican-Republic               70
Vietnam                          67
Guatemala                        64
Japan                            62
Poland                           60
Columbia                         59
Taiwan                           51
Haiti                            44
Iran                             43
Portugal                         37
Nicaragua                        34
Peru                             31
France                           29
Greece                      

### Option 1: most common value

In [ ]:
df2 = df.fillna(df.mode().iloc[0])

### Option 2: conditional most common

E.g. fill missing occupation by most common occupation with same education and sex

In [ ]:
x = df[(df['education'] == '10th') & (df['sex'] == 'Female')]['occupation']

In [ ]:
pd.pivot_table(data=df, index='education', columns='sex', values='occupation',
               aggfunc=lambda x: x.isna().sum())

sex,Female,Male
education,,
10th,42,60
11th,60,59
12th,18,22
1st-4th,3,9
5th-6th,10,20
7th-8th,23,50
9th,24,27
Assoc-acdm,20,27
Assoc-voc,39,22


In [ ]:
pd.pivot_table(data=df, index='education', columns='sex', values='occupation',
               aggfunc=lambda x: x.mode().iloc[0])

sex,Female,Male
education,,
10th,Other-service,Craft-repair
11th,Other-service,Craft-repair
12th,Other-service,Craft-repair
1st-4th,Other-service,Craft-repair
5th-6th,Machine-op-inspct,Other-service
7th-8th,Other-service,Craft-repair
9th,Other-service,Craft-repair
Assoc-acdm,Adm-clerical,Craft-repair
Assoc-voc,Adm-clerical,Craft-repair


In [ ]:
def most_common(dataframe):
    return dataframe.mode().iloc[0]

In [ ]:
filled_occ = df.groupby(['education', 'sex'], group_keys=False)['occupation'].apply(lambda x: x.fillna(most_common(x)))

In [ ]:
filled_occ

0             Adm-clerical
1          Exec-managerial
2        Handlers-cleaners
3        Handlers-cleaners
4           Prof-specialty
               ...        
32556         Tech-support
32557    Machine-op-inspct
32558         Adm-clerical
32559         Adm-clerical
32560      Exec-managerial
Name: occupation, Length: 32561, dtype: object

In [ ]:
filled_occ[df['occupation'].isna()]

27         Craft-repair
61         Craft-repair
69         Craft-repair
77         Craft-repair
106       Other-service
              ...      
32530    Prof-specialty
32531    Prof-specialty
32539    Prof-specialty
32541      Adm-clerical
32542      Craft-repair
Name: occupation, Length: 1843, dtype: object

In [ ]:
df[df['occupation'].isna()]['occupation']

27       NaN
61       NaN
69       NaN
77       NaN
106      NaN
        ... 
32530    NaN
32531    NaN
32539    NaN
32541    NaN
32542    NaN
Name: occupation, Length: 1843, dtype: object

In [ ]:
df3 = df.copy()
df3['occupation'] = filled_occ

In [ ]:
df3.isna().sum()

age                  0
workclass         1836
fnlwgt               0
education            0
education-num        0
marital-status       0
occupation           0
relationship         0
race                 0
sex                  0
capital-gain         0
capital-loss         0
hours-per-week       0
native-country     583
income               0
dtype: int64

### Option 3: Linear regression

In [ ]:
X = df3.drop(columns=['native-country', 'income'])


In [ ]:
X.shape

(32561, 13)

In [ ]:
train_mask = ~X.isna().any(axis=1)


In [ ]:
train_mask.sum()

30725

In [ ]:
X = pd.get_dummies(X.drop(columns=['workclass']))
X.shape

(32561, 56)

In [ ]:
X_train = X.loc[train_mask]
X_train.shape

(30725, 56)

In [ ]:
y_train = df3.loc[train_mask, 'workclass'].dropna()
y_train.shape

(30725,)

In [ ]:
X_inference = X[~train_mask]
X_inference.shape

(1836, 56)

In [ ]:
from sklearn.preprocessing import LabelEncoder


X = df3.drop(columns=['native-country', 'income'])

train_mask = ~X.isna().any(axis=1)

X = pd.get_dummies(X.drop(columns=['workclass']))

X_train = X.loc[train_mask]
X_inference = X[~train_mask]

enc = LabelEncoder()
y_train = enc.fit_transform(df3.loc[train_mask, 'workclass'].dropna())

In [ ]:
from sklearn.ensemble import RandomForestClassifier

preds_raw = RandomForestClassifier().fit(X_train, y_train).predict(X_inference)
preds = enc.inverse_transform(preds_raw)

In [ ]:
df3.loc[~train_mask, 'workclass'] = preds

In [ ]:
df3.isna().sum()

age                 0
workclass           0
fnlwgt              0
education           0
education-num       0
marital-status      0
occupation          0
relationship        0
race                0
sex                 0
capital-gain        0
capital-loss        0
hours-per-week      0
native-country    583
income              0
dtype: int64

In [ ]:
df3 = df3.fillna(df3.mode().iloc[0])

## 3. Encoding

In [ ]:
df3.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

### Label

In [ ]:
df3['income'] = df3['income'].map({'<=50K': 0, '>50K': 1})

### Sex

In [ ]:
df3['income'] = df3['income'].map({'Male': 0, 'Female': 1})

### Native country

In [ ]:
df3['native-country'] = df3['native-country'].apply(lambda x: 1 if x == 'United-States' else 0)

In [ ]:
df3.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
       'income'],
      dtype='object')

### Remaining columns

Make some groupings and leave for get_dummies

'relationship', 'workclass', 'marital-status', 'occupation', 'race'


## Scaling